# Egocentric Network analysis

The data: sample output from limesurvey
Ego's were ask to state their age and gender (1 = m, 2 = w), as well as to name their friends (alter 1 - n), whether these friends know each other and how well (1-3), their friends age and gender.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mlp
from pylab import text

In [2]:
# sample data with 3 egos
df = pd.DataFrame({'egoID': [1,2,3], 
                   'egoAge': [20,18,23], 
                   'egoGender': [1,2,2],
                  'Netsize': [4,3,5],'AlterAge1': [20,22,30],'AlterAge2': [18,26,20],'AlterAge3': [25,24,21],
                   'AlterAge4': [24,np.NaN,22], 'AlterAge5': [np.NaN,np.NaN,23],
                   'knowing1-2': [1,1,3],'knowing1-3': [2,1, np.NaN], 'knowing1-4': [1,1,1], 'knowing1-5': [1,2,1], 
                   'knowing2-3': [np.NaN,3,2], 'knowing2-4': [np.NaN,3,2], 'knowing2-5': [3,np.NaN,1],
                   'knowing3-4': [2, np.NaN,1], 'knowing3-5': [2,1,1], 'knowing4-5': [3,3,np.NaN]})
df

,AlterAge1,AlterAge2,AlterAge3,AlterAge4,AlterAge5,Netsize,egoAge,egoGender,egoID,knowing1-2,knowing1-3,knowing1-4,knowing1-5,knowing2-3,knowing2-4,knowing2-5,knowing3-4,knowing3-5,knowing4-5
0,20,18,25,24.0,NaN,4,20,1,1,1,2.0,1,1,NaN,NaN,3.0,2.0,2,3.0
1,22,26,24,NaN,NaN,3,18,2,2,1,1.0,1,2,3.0,3.0,NaN,NaN,1,3.0
2,30,20,21,22.0,23.0,5,23,2,3,3,NaN,1,1,2.0,2.0,1.0,1.0,1,NaN


This format is not appropriate for the desired analysis. Thus, I have to reshape it from wide to long format as well as create datasets, one for ego and alters attribute and one for the alter-alter ties.

## Alter-Alter ties

In [4]:
df_long = pd.wide_to_long(df, stubnames='knowing', i='egoID', j='alter-alter').reset_index()
df_long_matrix = df_long[['egoID', 'Netsize', 'alter-alter', 'knowing']].copy()
df_long_matrix.head()

,egoID,Netsize,alter-alter,knowing
0,1,4,1-2,1.0
1,2,3,1-2,1.0
2,3,5,1-2,3.0
3,1,4,1-3,2.0
4,2,3,1-3,1.0
5,3,5,1-3,NaN
6,1,4,1-4,1.0
7,2,3,1-4,1.0
8,3,5,1-4,1.0
9,1,4,1-5,1.0


In [5]:
df_long_matrix[['Source', 'Target']] = df_long_matrix['alter-alter'].str.split('-', expand=True, n=1)
df_long_matrix.sort_values(by=['egoID'], ascending=[True], inplace = True)
df_long_matrix.head()

,egoID,Netsize,alter-alter,knowing,Source,Target
0,1,4,1-2,1.0,1,2
18,1,4,2-5,3.0,2,5
15,1,4,2-4,NaN,2,4
12,1,4,2-3,NaN,2,3
24,1,4,3-5,2.0,3,5
9,1,4,1-5,1.0,1,5
21,1,4,3-4,2.0,3,4
3,1,4,1-3,2.0,1,3
27,1,4,4-5,3.0,4,5
6,1,4,1-4,1.0,1,4


In [6]:
# rows entfernen, in denen matrix == 0 und nicht NaN ist
alter_alter_df = df_long_matrix[df_long_matrix.knowing.notnull()].query('knowing != 0').copy()
alter_alter_df.head()

,egoID,Netsize,alter-alter,knowing,Source,Target
0,1,4,1-2,1.0,1,2
18,1,4,2-5,3.0,2,5
24,1,4,3-5,2.0,3,5
9,1,4,1-5,1.0,1,5
21,1,4,3-4,2.0,3,4
3,1,4,1-3,2.0,1,3
27,1,4,4-5,3.0,4,5
6,1,4,1-4,1.0,1,4
7,2,3,1-4,1.0,1,4
25,2,3,3-5,1.0,3,5


## Ego-Alter attributes